In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
import scipy.io as spi
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
import sys

C:\Users\Prahita\Anaconda2\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
#read train, user-features and item-features matrices and sparsify them
import pandas as pd
train = pd.read_csv("G:\\PGDBA\\Kaggle\\DSG2017\\data\\train.csv")
test = pd.read_csv("G:\\PGDBA\\Kaggle\\DSG2017\\data\\test.csv")
#user_item= pd.read_csv("C:\\Users\\pkhande5\\Desktop\\Kaggle\\deezer_sample.csv")
#user_item=  sps.csr_matrix(user_item, dtype=np.int16) 
# Save the matrix. They are already in csr format
#spi.mmwrite("user_item.mtx", user_item) 
        
user_feat = pd.read_csv("G:\\PGDBA\\Kaggle\\DSG2017\\data\\user_features.csv")
song_feat = pd.read_csv("G:\\PGDBA\\Kaggle\\DSG2017\\data\\item_features.csv")
song_feat['song_hash_id']= range(song_feat.shape[0])

In [3]:
all_data= pd.concat([train, test])
all_data=all_data.ix[:, ['user_id', 'media_id']]

In [8]:
#create a hash of indexes
media_index= {key: value for (value, key) in enumerate(song_feat['media_id'].values)}

In [ ]:
#uf['index']= [user_index[k] for k in uf['user_id'].values]
#mf['index']=[media_index[k] for k in mf['media_id'].values]

In [ ]:
#convert train to user-item matrix # Build the user x item matrices using scipy lil_matrix 
n_row= len(all_data['user_id'].unique())
n_col= len(all_data['media_id'].unique())

user_item_mat = sps.lil_matrix(((n_row), (n_col)), dtype=np.int8)
for i in range(len(train)): 
    sys.stdout.write( "\rProcessing row " + str(i) + "/ " + str(train.shape[0])) 
    sys.stdout.flush() 
    ui, mi = train["user_id"].values[i],  media_index[train["media_id"].values[i]] 
    user_item_mat[ui, mi] = train["is_listened"].values[i]

In [ ]:
mf_mat= sps.csr_matrix(mf, dtype=np.int16)
uf_mat= sps.csr_matrix(uf, dtype=np.int16)

In [ ]:
model.fit(user_item_mat,
          user_features=uf_mat,
          item_features=mf_mat,
          epochs=20)

In [ ]:
predictions = model.predict(uf['index'],
                            mf['index'],
                            user_features=uf_mat,
                            item_features=mf_mat)

In [ ]:
def sample_recommendation(model, data, user_ids):

    #number of users and movies in training data
    n_users, n_items = data['train'].shape

    #generate recommendations for each user we input
    for user_id in user_ids:

        #movies they already like
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        #movies our model predicts they will like
        scores = model.predict(user_id, np.arange(n_items))
        #rank them in order of most liked to least
        top_items = data['item_labels'][np.argsort(-scores)]

        #print out the results
        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)